# Optimising Demand Response Strategies for Carbon Intelligent Load Shifting

# Investigating Data Size and Types

**NOTEBOOK PURPOSE(S):**
* Explore methods for reducing the overall size of data used in this project, as well as more efficient methods of operating on the data.

**NOTEBOOK OUTPUTS:**
* N/A testing only - all temporaryoutputs should be deleted by running the appropriate cells

A note on measuring memory usage

In reviewing how exactly to measure memory usage and process usage, I consulted the following resources:
* https://psutil.readthedocs.io/en/latest/
* https://docs.python.org/3/library/tracemalloc.html
* https://pythonspeed.com/articles/measuring-memory-python/
* https://www.geeksforgeeks.org/python/monitoring-memory-usage-of-a-running-python-program/
* https://techkoalainsights.com/7-advanced-python-memory-profiling-techniques-for-production-debugging-beyond-basic-tools-cfe90589333f

### Importing Libraries

In [1]:
%matplotlib inline

# ────────────────────────────────────────────────────────────────────────────
# Data Manipulation & Analysis
# ─────────────────────────────────────────────────────────────────────────────
import pandas as pd
import polars as pl

# ─────────────────────────────────────────────────────────────────────────────
# Geospatial Data Handling
# ─────────────────────────────────────────────────────────────────────────────
from shapely.wkb import loads

# ─────────────────────────────────────────────────────────────────────────────
# Notebook/Display Tools
# ─────────────────────────────────────────────────────────────────────────────
from IPython.display import display

# ─────────────────────────────────────────────────────────────────────────────
# System / Miscellaneous
# ─────────────────────────────────────────────────────────────────────────────
import os
import binascii
import time
import tracemalloc
import psutil

## Loading Data from Local Storage

In [2]:
root_directory = os.path.join('..', '..')
base_data_directory = os.path.join(root_directory, "data")
hitachi_data_directory = os.path.join(base_data_directory, 'hitachi')
meter_reading_directory = os.path.join(hitachi_data_directory, "meter_primary_files")  # Directory for meter readings

print("\n" + "-" * 120)
print(f"Contents of '{base_data_directory}' and subdirectories:\n" + "-" * 120)
for root, dirs, files in os.walk(base_data_directory):

    dirs.sort()  # Sort directories for consistent output
    files.sort()  # Sort files for consistent output

    for f in files:
        rel_dir = os.path.relpath(root, base_data_directory)
        rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
        print(f"  - {rel_file}")


------------------------------------------------------------------------------------------------------------------------
Contents of '../../data' and subdirectories:
------------------------------------------------------------------------------------------------------------------------
  - .DS_Store
  - era5/.DS_Store
  - era5/grib_downloads/125ae282169904325e8bc153160be150.grib
  - era5/grib_downloads/125ae282169904325e8bc153160be150.grib.47d85.idx
  - era5/grib_downloads/289f2aac241f8a158ff074a66682452e.grib
  - era5/grib_downloads/554832a6209258041784298e5401a7ab.grib
  - era5/grib_downloads/5aee58993569287064988fbc8ad385dd.grib
  - era5/grib_downloads/5bcc58c42bdde8ce6b147b00099404bc.grib
  - era5/grib_downloads/ad36c26a5d6daae43c9aeab1747e078c.grib
  - era5/grib_downloads/b4eac1bff8a020500806be638e9d4ab9.grib
  - era5/grib_downloads/bc20f736fa82ab5167820d9116ab4859.grib
  - era5/grib_downloads/c8a985ffc4908e6597c4498ff596cbad.grib
  - era5/grib_downloads/d1313a3f750d6e7bd89dff34b

In [3]:
# File names copied from the output of the above code block
# These are the files we will use to investigate the data size reduction.
customers_path = os.path.join(hitachi_data_directory, "customers_20250714_1401.parquet")
grid_readings_path = os.path.join(hitachi_data_directory, "grid_readings_20250714_1401.parquet")

# This file is quite large (~8 GB), so will be a good test case for large files
meter_readings_2021_path = os.path.join(meter_reading_directory, "meter_readings_2021_20250714_2015.parquet")
# This file is also large, but only ~250MB, so it will be a good test case for smaller files
meter_readings_2022_path = os.path.join(meter_reading_directory, "meter_readings_2022_20250714_2324.parquet")
weather_path = os.path.join(hitachi_data_directory, "weather_20250714_1401.parquet")

In [4]:
# Size of files on disk
print("\n" + "-" * 120)
print(f"Size of files on disk:\n" + "-" * 120)
print(f"\tCustomers table: {os.path.getsize(customers_path) / (1024 * 1024):,.2f} MB")
print(f"\tGrid readings table: {os.path.getsize(grid_readings_path) / (1024 * 1024):,.2f} MB")
print(f"\tMeter readings table 2021: {os.path.getsize(meter_readings_2021_path) / (1024 * 1024):,.2f} MB")
print(f"\tMeter readings table 2022: {os.path.getsize(meter_readings_2022_path) / (1024 * 1024):,.2f} MB")
print(f"\tWeather table: {os.path.getsize(weather_path) / (1024 * 1024):,.2f} MB")


------------------------------------------------------------------------------------------------------------------------
Size of files on disk:
------------------------------------------------------------------------------------------------------------------------
	Customers table: 3.80 MB
	Grid readings table: 21.71 MB
	Meter readings table 2021: 2,080.60 MB
	Meter readings table 2022: 10,335.06 MB
	Weather table: 77.77 MB


We'll now load and analyse each dataset separately. 

The kernel may need to be restarted betweeen analyses as the datasets are so large.

We will also attempt to time processing to see the impact of using polars' 'lazy' implementation.

## Reducing Data Size

### Helper Functions

In [5]:
def memory_usage_per_column(df: pl.DataFrame) -> pl.DataFrame:
    """
    Computes the memory usage per column in a Polars DataFrame.

    Parameters
    ----------
    df : pl.DataFrame
        The Polars DataFrame for which to compute memory usage.
    Returns
    -------
    pl.DataFrame
        A DataFrame with memory usage per column:
        - column:           column name
        - bytes:            memory usage in bytes
        - mb:               memory usage in megabytes
        - percent_of_total: percentage of total memory usage
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("Input must be a Polars DataFrame")
    if df.is_empty():
        return pl.DataFrame(columns=["column", "bytes", "mb", "percent_of_total"])
    else:
        total_bytes = df.estimated_size()
        rows = []
        for col in df.columns:
            col_bytes = df[col].estimated_size()
            mb = col_bytes / (1024**2)
            pct = (col_bytes / total_bytes * 100) if total_bytes else 0.0
            rows.append((col, col_bytes, mb, pct))
        return pl.DataFrame(rows, schema=[
            "column", "bytes", "mb", "percent_of_total"
        ],
        orient="row").sort("bytes", descending=True)

In [6]:
def dataframe_summary(
        df: pl.DataFrame,
        name: str = None,
        filepath: str=None
) -> None:
    """
    Prints a three-part summary of a provided polars dataframe including
    Schema, Per-column memory usage, and the first 5 rows of data

    Parameters
    ----------
    df : pl.DataFrame
        The DataFrame to summarise
    name : str, optional
        An optional name for the DataFrame, which will be included in the title of the summary
    filepath : str, optional
        An optional file path to the DataFrame, which will used to display the size of the file on disk.
        If provided, the file size will be included in the summary.

    Returns
    -------
    None
        Prints the summary directly to the console.
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("Input must be a Polars DataFrame")

    if df.is_empty():
        print("DataFrame is empty.")
        return

    title = f"DataFrame Summary{' — ' + name if name else ''} "
    print("\n" + "-" * 120)
    print(title + "\n" + "-"* 120)

    if filepath:
        file_size_disk_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"\nFile size on disk: {file_size_disk_mb:,.2f} MB")

    file_size_mem_mb = df.estimated_size() / (1024 * 1024)
    print(f"\nEstimated size in memory: {file_size_mem_mb:,.2f} MB")

    print("\nPer-column memory usage:\n"+ "-" * 120)
    mem_df = memory_usage_per_column(df)
    # pretty-print the little table
    for row in mem_df.iter_rows(named=True):
        print(f"  • {row['column']:<15s} {row['mb']:6.2f} MB  ({row['percent_of_total']:5.1f}%)")

    print("\nSchema:\n" + "-" * 120)
    for col, dtype in df.schema.items():
        print(f"  • {col}: {dtype}")

    print("\nFirst 5 rows: \n" + "-" * 120)

    # redact id and location columns if present (id, ca_id, location, customer_longitude, customer_latitude) to preserve confidentiality
    redact_cols = ['id', 'ca_id', 'value', 'location', 'customer_longitude', 'customer_latitude']
    for col in redact_cols:
        if col in df.columns:
            df = df.drop(col)

    display(df.head(5))
    print("Note: id and location related columns are redacted here to preserve customer confidentiality")
    return None


In [7]:
def wkb_to_coords(hex_wkb: str):
    """
    Convert a hex WKB string to coordinates (x, y).
    Uses binascii to decode the hex string and shapely's load function to convert WKB to a Point object.

    Parameters:
    ----------
    hex_wkb : str
        Hexadecimal string representing the WKB (Well-Known Binary) format of a point.

    Returns:
    -------
    tuple
        A tuple containing the x and y coordinates of the point, or (None, None)
        if the conversion fails.
    """
    try:
        point = loads(binascii.unhexlify(hex_wkb))
        return point.x, point.y
    except Exception:
        return None, None


##### Single Use Functions

In [8]:
def memory_usage_per_column_pd(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes the memory usage per column in a Pandas DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        The Pandas DataFrame for which to compute memory usage.

    Returns
    -------
    pd.DataFrame
        A DataFrame with memory usage per column:
        - column:           column name
        - bytes:            memory usage in bytes
        - mb:               memory usage in megabytes
        - percent_of_total: percentage of total memory usage
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a Pandas DataFrame")

    if df.empty:
        return pd.DataFrame(columns=["column", "bytes", "mb", "percent_of_total"])

    mem_usage = df.memory_usage(deep=True)
    total_bytes = mem_usage.sum()

    result = pd.DataFrame({
        "column": mem_usage.index,
        "bytes": mem_usage.values,
        "mb": mem_usage.values / (1024 ** 2),
        "percent_of_total": (mem_usage.values / total_bytes) * 100
    }).reset_index(drop=True)

    return result.sort_values("bytes", ascending=False)

In [9]:
def dataframe_summary_pd(
        df: pd.DataFrame,
        name: str = None,
        filepath: str = None
) -> None:
    """
    Prints a three-part summary of a provided pandas dataframe including
    Schema, Per-column memory usage, and the first 5 rows of data.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame to summarize
    name : str, optional
        An optional name for the DataFrame
    filepath : str, optional
        An optional file path to include file size information

    Returns
    -------
    None
        Prints the summary to console
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a Pandas DataFrame")

    if df.empty:
        print("DataFrame is empty.")
        return

    title = f"DataFrame Summary{' — ' + name if name else ''} "
    print("\n" + "-" * 120)
    print(title + "\n" + "-"* 120)

    if filepath:
        file_size_disk_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"\nFile size on disk: {file_size_disk_mb:,.2f} MB")

    file_size_mem_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
    print(f"\nEstimated size in memory: {file_size_mem_mb:,.2f} MB")

    print("\nPer-column memory usage:\n"+ "-" * 120)
    mem_df = memory_usage_per_column_pd(df)
    for _, row in mem_df.iterrows():
        print(f"  • {row['column']:<15s} {row['mb']:6.2f} MB  ({row['percent_of_total']:5.1f}%)")

    print("\nSchema:\n" + "-" * 120)
    for col, dtype in df.dtypes.items():
        print(f"  • {col}: {dtype}")

    print("\nFirst 5 rows: \n" + "-" * 120)

    # redact id and location columns if present (id, ca_id, location, customer_longitude, customer_latitude) to preserve confidentiality
    redact_cols = ['id', 'ca_id', 'value','location', 'customer_longitude', 'customer_latitude']
    for col in redact_cols:
        if col in df.columns:
            df = df.drop(columns=[col])
    print(df.head(5))
    print("Note: id and location related columns are redacted here to preserve customer confidentiality")
    return None

### General Use of Parquet Files

Polars and Parquet were adopted from the onset of this analysis due to their superior performance with large datasets.

Parquet works with a columnar binary format which compresses data (deduplicating repeated values), unlike CSV's which uses a row-based text format that ends up wasting space on delimiters, and also forces all data to be parsed as strings.
As a result, parquet files load faster, take up less space on both disk and memory, and (as is shown later in this notebook), polars' lazy execution drastically reduces execution costs.

Below, we demonstrate the advantage of Polars and Parquet files over Pandas and CSV.

In [10]:
t1 = time.time()
customers_table_pldf = pl.read_parquet(customers_path)
t2 = time.time()
print(f"Time to read customers parquet file with Polars: {t2 - t1:.2f} seconds")

test_customers_table_df = customers_table_pldf.to_pandas()

test_customers_table_filepath = os.path.join(hitachi_data_directory, "customers_20250714_1401.csv")

# including no index for comparability because polars parquet files do not retain them
test_customers_table_df.to_csv(test_customers_table_filepath, index=False)

print("\n")
t3 = time.time()
test_customers_table_df = pd.read_csv(test_customers_table_filepath)
t4 = time.time()
print(f"Time to read customers csv file with Pandas: {t4 - t3:.2f} seconds\n\n")

dataframe_summary(
    customers_table_pldf,
    name="Customers Table - Polars to Parquet",
    filepath=customers_path)


dataframe_summary_pd(
    test_customers_table_df,
    name="Customers Table - Pandas to CSV",
    filepath=os.path.join(hitachi_data_directory, "customers_20250714_1401.csv")
)

Time to read customers parquet file with Polars: 0.02 seconds


Time to read customers csv file with Pandas: 0.14 seconds



------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table - Polars to Parquet 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.80 MB

Estimated size in memory: 14.28 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location         10.76 MB  ( 75.4%)
  • id                2.40 MB  ( 16.8%)
  • city              1.11 MB  (  7.8%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: String
  • location: String
  • city: String

First 5 rows: 
----------------------------------------------

city
str
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table - Pandas to CSV 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 14.92 MB

Estimated size in memory: 38.13 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location         23.03 MB  ( 60.4%)
  • city             13.38 MB  ( 35.1%)
  • id                1.72 MB  (  4.5%)
  • Index             0.00 MB  (  0.0%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: float64
  • location: object
  • city: object

First 5 rows: 
---------------------------------------

In [11]:
# Deleting the CSV file as it was just for comparison purposes

new_filepath = os.path.join(hitachi_data_directory, "customers_20250714_1401.csv")

try:
    os.remove(new_filepath)
    print(f"Successfully deleted: {new_filepath}")
except FileNotFoundError:
    print(f"File does not exist: {new_filepath}")
except PermissionError:
    print(f"Permission denied: Cannot delete {new_filepath}")
except Exception as e:
    print(f"Error deleting file: {e}")

Successfully deleted: ../../data/hitachi/customers_20250714_1401.csv


### Individual Files

#### Customers table [customers_table_pldf]

In [12]:
# Loading the dataframe
customers_table_pldf = pl.read_parquet(customers_path)

In [13]:
dataframe_summary(customers_table_pldf, name="Customers Table", filepath=customers_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.80 MB

Estimated size in memory: 14.28 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location         10.76 MB  ( 75.4%)
  • id                2.40 MB  ( 16.8%)
  • city              1.11 MB  (  7.8%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: String
  • location: String
  • city: String

First 5 rows: 
------------------------------------------------------------------------------------------------------------------------


city
str
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality


Based on the above - all three values are currently stored as string, which is certainly not the most efficient form of storage.
Lets consider the options.

1. For the id column, we could change the id to integer, although this is not good practice to store ids as integers. Also as it is only ~17% of the overall storage costs, this is something we can potentially deprioritise in favor of other reductions.
2. For the location column, this is stored as a very long string. The location information is essential, but it may not be entirely necessary to store as a string. We may look into storing as longitude and latitude coordinates.
3. For the city column, these are stored as strings, but are relatively short and so not taking up very much space.**NOTE:** <span style="color:#d62728"> The analysis noted here has been redacted. </span> We could potentially look into dropping this column or changing the type to boolean, categorical, or integer.

Since the location is the largest - lets start with this one.

##### 'location'

Let's calculate the longitude and latitude for the location column, drop the long string value, and then experiment with the float64 v float32 datatypes

In [14]:
# Calculating longitude and latitude from the location column
# Method 1 - converting to pandas DataFrame, using lambda function

# Start memory tracing
proc = psutil.Process()
mem_before = proc.memory_info().rss
tracemalloc.start()

# Start timing the conversion
t0 = time.perf_counter()

# First, convert the Polars DataFrame to a Pandas DataFrame
customers_pddf = customers_table_pldf.to_pandas()

# Decode the 'location' hex values into latitude and longitude coordinates
# and add them as new columns 'lon' and 'lat' in both DataFrames
customers_pddf[["customer_longitude", "customer_latitude"]] = customers_pddf["location"].apply(
    lambda x: pd.Series(wkb_to_coords(x))
)

# Convert the Pandas DataFrame back to a Polars DataFrame
customers_table_pldf = pl.from_pandas(customers_pddf)

# Stop timer and memory tracing
t1 = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
mem_after = proc.memory_info().rss

# create temporary file path for the updated customers table
customers_path_temp = customers_path.replace(".parquet", "_temp.parquet")
# Save the updated DataFrame to a new Parquet file
customers_table_pldf_copy = customers_table_pldf.drop("location")
customers_table_pldf_copy.write_parquet(customers_path_temp)

# Return stats
print(f"Time taken to convert 'location' to 'lon' and 'lat': {t1 - t0:.3f} seconds")
print(f"PyObject Memory usage [tracemalloc]: {current / (1024 * 1024):.2f} MB (peak: {peak / (1024 * 1024):.2f} MB)")
print(f"Total Physical Memory usage (RSS) [psutil]: {(mem_after - mem_before) / (2**20):.3f} MB")
dataframe_summary(customers_table_pldf_copy, name="Customers Table with Coordinates", filepath=customers_path_temp)


Time taken to convert 'location' to 'lon' and 'lat': 25.903 seconds
PyObject Memory usage [tracemalloc]: 42.64 MB (peak: 636.23 MB)
Total Physical Memory usage (RSS) [psutil]: 671.016 MB

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table with Coordinates 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.69 MB

Estimated size in memory: 6.96 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • id                2.40 MB  ( 34.5%)
  • customer_longitude   1.72 MB  ( 24.7%)
  • customer_latitude   1.72 MB  ( 24.7%)
  • city              1.11 MB  ( 16.0%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: Str

city
str
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality


In [15]:
# Method 2 - using Polars lazy API for better performance on large datasets
# We will time this execution to see how it compares to the previous method
# We will also monitor the memory usage of the DataFrame before and after the operation

# Start memory tracing
proc = psutil.Process()
mem_before = proc.memory_info().rss
tracemalloc.start()

# Start timing the conversion
t0 = time.perf_counter()
# build a lazy pipeline:
lf = (
    # scan the Parquet file
    pl.scan_parquet(customers_path)
    # map the 'location' column from hex WKB to coordinates
    .with_columns([
        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[0], return_dtype=pl.Float64)
          .alias("customer_longitude"),

        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[1], return_dtype=pl.Float64)
          .alias("customer_latitude"),
    ])
)
# execute, returning an eager DataFrame
df = lf.collect()

# Stop timer and memory tracing
t1 = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
mem_after = proc.memory_info().rss


customers_path_temp_lf = customers_path.replace(".parquet", "_temp_lf.parquet")
customers_table_pldf_copy_lf = df.drop("location")
# Save the updated DataFrame to a new Parquet file
customers_table_pldf_copy_lf.write_parquet(customers_path_temp_lf)

# Return stats
print(f"Time taken to convert 'location' to 'lon' and 'lat': {t1 - t0:.3f} seconds")
print(f"PyObject Memory usage [tracemalloc]: {current / (1024 * 1024):.2f} MB (peak: {peak / (1024 * 1024):.2f} MB)")
print(f"Total Physical Memory usage (RSS) [psutil]: {(mem_after - mem_before) / (2**20):.3f} MB")

dataframe_summary(customers_table_pldf_copy_lf, name="Customers Table with Coordinates", filepath=customers_path_temp_lf)

# look at the how the lazyframe works
print(lf.explain())

Time taken to convert 'location' to 'lon' and 'lat': 8.998 seconds
PyObject Memory usage [tracemalloc]: 0.13 MB (peak: 0.14 MB)
Total Physical Memory usage (RSS) [psutil]: 24.391 MB

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table with Coordinates 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.69 MB

Estimated size in memory: 6.96 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • id                2.40 MB  ( 34.5%)
  • customer_longitude   1.72 MB  ( 24.7%)
  • customer_latitude   1.72 MB  ( 24.7%)
  • city              1.11 MB  ( 16.0%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: String
 

city
str
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality
 WITH_COLUMNS:
 [col("location").map_list().alias("customer_longitude"), col("location").map_list().alias("customer_latitude")] 
  Parquet SCAN [../../data/hitachi/customers_20250714_1401.parquet] [id: 6324863072]
  PROJECT */3 COLUMNS


In [16]:
print("Note: id and location related columns are redacted here to preserve customer confidentiality")
print(customers_table_pldf.select(pl.exclude("id", "location", "customer_longitude", "customer_latitude")).describe())

Note: id and location related columns are redacted here to preserve customer confidentiality
shape: (9, 2)
┌────────────┬────────┐
│ statistic  ┆ city   │
│ ---        ┆ ---    │
│ str        ┆ str    │
╞════════════╪════════╡
│ count      ┆ 225683 │
│ null_count ┆ 0      │
│ mean       ┆ null   │
│ std        ┆ null   │
│ min        ┆ delhi  │
│ 25%        ┆ null   │
│ 50%        ┆ null   │
│ 75%        ┆ null   │
│ max        ┆ mumbai │
└────────────┴────────┘


Based on the above we have determined two things:
1. It will be more efficient to store the customer location in two separate columns
2. It is significantly faster and much less memory intensive to use the lazy api in polars.

Also, after looking a the range of values that we are left with in longitude and latitude (out to 6 decimal points), it would likely be safer to leave these columns as Float64 type to retain their precision. Float32 retains precision out to 7 decimal points, but we do not know if we will require further precision present in Float64 later on. Best to air on the side of caution, especially with this smaller file (now even moreso)

The following article contains some information about the precision and significant digits of different data types: https://massedcompute.com/faq-answers/?question=How%20does%20float32%20precision%20compare%20to%20float64%20precision%20in%20terms%20of%20accuracy%20and%20speed%20for%20machine%20learning%20tasks?#:~:text=Accuracy%20Comparison&text=Float32%20offers%20about%207%20significant,numerical%20instability%20in%20edge%20cases.

We'll apply the changes in full processing later on, for now lets clean up the extra files we just created.

In [17]:
# Removing the temporary files created during the process
try:
    os.remove(customers_path_temp)
    print(f"Successfully deleted: {customers_path_temp}")

except FileNotFoundError:
    print(f"File does not exist: {customers_path_temp}")

try:
    os.remove(customers_path_temp_lf)
    print(f"Successfully deleted: {customers_path_temp_lf}")

except FileNotFoundError:
    print(f"File does not exist: {customers_path_temp_lf}")

Successfully deleted: ../../data/hitachi/customers_20250714_1401_temp.parquet
Successfully deleted: ../../data/hitachi/customers_20250714_1401_temp_lf.parquet


##### 'city'

We'll now look and see if we can reduce the size of the city column.

Since the lazy method proved to be more efficient, we'll continue with that.

In [18]:
# Dropping the modified columns
try:
    customers_table_pldf = customers_table_pldf.drop("customer_longitude", "customer_latitude")
except Exception as e:
    print(f"Error dropping columns: {e}")

# Display the first few rows of the DataFrame
print("Note: id and location related columns are redacted here to preserve customer confidentiality")
print(customers_table_pldf.head().drop("id", "location"))

Note: id and location related columns are redacted here to preserve customer confidentiality
shape: (5, 1)
┌────────┐
│ city   │
│ ---    │
│ str    │
╞════════╡
│ mumbai │
│ mumbai │
│ mumbai │
│ mumbai │
│ mumbai │
└────────┘


In [19]:
# Start memory tracing
proc = psutil.Process()
mem_before = proc.memory_info().rss
tracemalloc.start()

# Start timing the conversion
t0 = time.perf_counter()
# build a lazy pipeline:
lazy_frame = (
    # scan the Parquet file
    pl.scan_parquet(customers_path)
    # map the 'location' column from hex WKB to coordinates
    .with_columns([
        # converting to categorical data type
        pl.col("city").cast(pl.Categorical).alias("city_category"),
        # converting to boolean data type
        (pl.col("city")=="delhi").alias("city_is_delhi"),
        # converting to UInt8 data type - requires pass through of categorical data type
        pl.col("city").cast(pl.Categorical).cast(pl.UInt32).cast(pl.UInt8).alias("city_uint8"),
    ])
)
df = lazy_frame.collect()

# Stop timer and memory tracing
t1 = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
mem_after = proc.memory_info().rss

# create temporary file path for the updated customers table
customers_path_city = customers_path.replace(".parquet", "_temp_city.parquet")
customers_table_pldf_city = df
# Save the updated DataFrame to a new Parquet file
customers_table_pldf_city.write_parquet(customers_path_city)

# Return stats
print(f"Time taken to convert 'city' to various datatypes: {t1 - t0:.2f} seconds")
print(f"PyObject Memory usage [tracemalloc]: {current / (1024 * 1024):.2f} MB (peak: {peak / (1024 * 1024):.2f} MB)")
print(f"Total Physical Memory usage (RSS) [psutil]: {(mem_after - mem_before) / (2**20):.3f} MB")

print("Note: id and location related columns are redacted here to preserve customer confidentiality")
dataframe_summary(customers_table_pldf_city, name="Customers Table with Varying 'city' datatypes", filepath=customers_path_city)


Time taken to convert 'city' to various datatypes: 0.02 seconds
PyObject Memory usage [tracemalloc]: 0.00 MB (peak: 0.01 MB)
Total Physical Memory usage (RSS) [psutil]: 30.812 MB
Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Customers Table with Varying 'city' datatypes 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.80 MB

Estimated size in memory: 15.38 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location         10.76 MB  ( 70.0%)
  • id                2.40 MB  ( 15.6%)
  • city              1.11 MB  (  7.2%)
  • city_category     0.86 MB  (  5.6%)
  • city_uint8        0.22 MB  (  1.4%)
  • cit

city,city_category,city_is_delhi,city_uint8
str,cat,bool,u8
"""mumbai""","""mumbai""",false,0
"""mumbai""","""mumbai""",false,0
"""mumbai""","""mumbai""",false,0
"""mumbai""","""mumbai""",false,0
"""mumbai""","""mumbai""",false,0


Note: id and location related columns are redacted here to preserve customer confidentiality


In [20]:
# remove the temporary files created during the process
try:
    os.remove(customers_path_city)
    print(f"Successfully deleted: {customers_path_city}")
except FileNotFoundError:
    print(f"File does not exist: {customers_path_city}")

Successfully deleted: ../../data/hitachi/customers_20250714_1401_temp_city.parquet


Based on this - we'll likely store the city variable as categorical to retain the readability.

But it is good to know that boolean is available and could still significantly reduce the file size if needed later

##### Testing Implementation of Changes

Based on the contents of this article, I will continue to use the default 'snappy' compression, but also ensure to specify it for clarity

https://dev.to/alexmercedcoder/all-about-parquet-part-05-compression-techniques-in-parquet-4bcb

In [21]:
# We will change the location to two separate columns for longitude and latitude
# We will also convert the 'city' column to a categorical data type for better memory efficiency.
# We will then drop the unnecessary columns, and save the DataFrame to a new Parquet file()

# We'll also monitor the memory usage and time taken for this operation
# Start memory tracing
proc = psutil.Process()
mem_before = proc.memory_info().rss
tracemalloc.start()
# Start timing the conversion
t0 = time.perf_counter()

customers_lazy_operations = (
    pl.scan_parquet(customers_path)
    .with_columns([
        # converting to two separate columns for longitude and latitude
        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[0], return_dtype=pl.Float64)
          .alias("customer_longitude"),
        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[1], return_dtype=pl.Float64)
          .alias("customer_latitude"),

        # converting 'city' column to categorical data type
        pl.col("city").cast(pl.Categorical),
    ])
    .drop("location")  # drop the original 'location' column
)

customers_table_optimised_pldf = customers_lazy_operations.collect()

# Stop timer and memory tracing
t1 = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
mem_after = proc.memory_info().rss

# Return stats
print(f"Time taken to optimize 'location' and 'city' columns: {t1 - t0:.2f} seconds")
print(f"PyObject Memory usage [tracemalloc]: {current / (1024 * 1024):.2f} MB (peak: {peak / (1024 * 1024):.2f} MB)")
print(f"Total Physical Memory usage (RSS) [psutil]: {(mem_after - mem_before) / (2**20):.3f} MB")

Time taken to optimize 'location' and 'city' columns: 8.59 seconds
PyObject Memory usage [tracemalloc]: 0.01 MB (peak: 0.02 MB)
Total Physical Memory usage (RSS) [psutil]: 24.859 MB


In [22]:
# Save the modified DataFrame to a new Parquet file

# extracting base name
base, ext = os.path.splitext(customers_path)
customers_table_optimised_path = f"{base}_optimised{ext}"

# saving new dataframe
customers_table_optimised_pldf.write_parquet(
    customers_table_optimised_path,
    compression="snappy",  # Using snappy compression for better performance
    statistics=True,  # Enable statistics for better query performance (doesn't make a noticeable change on disk storage)
)

In [23]:
# View the difference between the original and optimized DataFrames
print("Note: id and location related columns are redacted here to preserve customer confidentiality")
dataframe_summary(customers_table_pldf, name="Original Customers Table", filepath=customers_path)
dataframe_summary(customers_table_optimised_pldf, name="Optimized Customers Table", filepath=customers_table_optimised_path)

Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Original Customers Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 3.80 MB

Estimated size in memory: 14.28 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location         10.76 MB  ( 75.4%)
  • id                2.40 MB  ( 16.8%)
  • city              1.11 MB  (  7.8%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: String
  • location: String
  • city: String

First 5 rows: 
----------------------------------------------------------------------------------------

city
str
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Optimized Customers Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 4.63 MB

Estimated size in memory: 6.71 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • id                2.40 MB  ( 35.8%)
  • customer_longitude   1.72 MB  ( 25.7%)
  • customer_latitude   1.72 MB  ( 25.7%)
  • city              0.86 MB  ( 12.8%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • id: String
  • city: Categorical(ordering='physical')
  • customer_longitude: Float64
  • customer_latitude:

city
cat
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""
"""mumbai"""


Note: id and location related columns are redacted here to preserve customer confidentiality


---

As we can see, the file size on disk may have increased slightly, but the file size in memory has been reduced by more than 50%.

---

In [24]:
# removing the temporary files created during the process
try:
    os.remove(customers_table_optimised_path)
    print(f"Successfully deleted: {customers_table_optimised_path}")
except FileNotFoundError:
    print(f"File does not exist: {customers_table_optimised_path}")

Successfully deleted: ../../data/hitachi/customers_20250714_1401_optimised.parquet


#### weather table [weather_table_pldf]

We now look at the weather table data.

In [25]:
# Loading the data
weather_table_pldf = pl.read_parquet(weather_path)

In [26]:
dataframe_summary(df=weather_table_pldf, name="Weather Table", filepath=weather_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Weather Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 77.77 MB

Estimated size in memory: 610.27 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location        190.24 MB  ( 31.2%)
  • wind_speed       61.35 MB  ( 10.1%)
  • wind_direction   61.35 MB  ( 10.1%)
  • temperature      61.35 MB  ( 10.1%)
  • precipitation    61.35 MB  ( 10.1%)
  • surface_net_solar_radiation  61.35 MB  ( 10.1%)
  • surface_solar_radiation_downwards  61.35 MB  ( 10.1%)
  • timestamp        30.44 MB  (  5.0%)
  • city             21.47 MB  (  3.5%)

Schema:
--------------------------------------------------------------------------------------------------------------

timestamp,wind_speed,wind_direction,temperature,city,precipitation,surface_net_solar_radiation,surface_solar_radiation_downwards
datetime[μs],"decimal[38,18]","decimal[38,20]","decimal[38,15]",str,"decimal[38,28]","decimal[38,17]","decimal[38,17]"
2024-12-13 13:00:00,8.013575020999754000,105.09231811866465000000,15.975219726562500,"""delhi""",0.0000000000000000000000000000,12296608.00000000000000000,14926616.00000000000000000
2024-12-13 14:00:00,8.289883957905610000,110.72614327536141000000,14.646026611328125,"""delhi""",0.0000000000000000000000000000,12108108.00000000000000000,14736050.00000000000000000
2024-12-13 14:00:00,8.336890954411203000,110.27725102589554000000,14.629241943359375,"""delhi""",0.0000000000000000000000000000,12149601.00000000000000000,14737817.00000000000000000
2024-12-13 14:00:00,8.403726468287198000,109.70666556656998000000,14.623779296875000,"""delhi""",0.0000000000000000000000000000,12240716.00000000000000000,14738703.00000000000000000
2024-12-13 14:00:00,8.454637190808407000,109.04352821446052000000,14.611297607421875,"""delhi""",0.0000000000000000000000000000,12278334.00000000000000000,14740205.00000000000000000


Note: id and location related columns are redacted here to preserve customer confidentiality


In [27]:
weather_table_pldf.describe()

statistic,timestamp,wind_speed,wind_direction,temperature,city,precipitation,location,surface_net_solar_radiation,surface_solar_radiation_downwards
str,str,f64,f64,f64,str,f64,str,f64,f64
"""count""","""3989664""",2.046285e6,2.046285e6,2.046285e6,"""3989664""",2.046285e6,"""3989664""",1.889953e6,1.889954e6
"""null_count""","""0""",1.943379e6,1.943379e6,1.943379e6,"""0""",1.943379e6,"""0""",2.099711e6,2.09971e6
"""mean""","""2022-09-15 12:30:00""",5.913636,167.302513,25.379801,null,0.001737,null,1.1276e7,1.3538e7
"""std""",null,3.125727,88.154022,6.977661,null,0.006866,null,6.9285e6,8.3022e6
"""min""","""2019-12-31 01:00:00""",0.005403,0.000265,0.487457,"""delhi""",0.0,"""0101000020E610000014AE47E17A24…",0.0,0.0
"""25%""","""2021-05-08 19:00:00""",3.790591,106.347651,21.677216,null,0.0,null,5.062334e6,6.017553e6
"""50%""","""2022-09-15 13:00:00""",5.431484,145.85406,26.418579,null,0.000001,null,1.2313436e7,1.4862e7
"""75%""","""2024-01-23 06:00:00""",7.45679,241.901807,29.746643,null,0.000466,null,1.6668687e7,1.9912196e7
"""max""","""2025-06-01 00:00:00""",42.517314,359.999979,46.374786,"""mumbai""",0.314005,"""0101000020E6100000F6285C8FC255…",2.49234e7,2.9038414e7


---

Though not a large file on the disk, this dataset takes up a comparatively large of memory (~8 times more than disk).

Repeated variables:
* 'location' - we have already seen this with the customer location data and know that it will likely reduce the size of the file to store this string value as individual longitude and latitude coordinates. We will double check this as well to be certain of its effectiveness.
* 'city' - again, we have already seen this and know that we can change the datatype to categorical. It may not save a large amount of memory, but it is a reduction.


New variables:

* 'wind_direction' - wind direction is recorded in degrees ranging from 0 to 360. It is doubtful that a thousandth of a degree would make much of a difference in our analysis as we are not predicting the distance or location that a particle will move in the air. We will assume that we can safely drop this precision from its existing Decimal(38,18) down to Float32 datatype. This will retain precision 7 digits after the decimal.
* 'wind_speed' - wind speed is also a variable whose extreme precision is unlikely to affect our analysis, so we will drop this to Float32 as well.
* 'temperature' - is surface temperature measured in degrees celsius and currently has datatype of Decimal(38,15), meaning up to 23 digits to the left of the decimal. While temperature is likely to have more of an impact on our analysis, we do not need more than 2 digits to the left of the decimal as temperatures are unlikely to go above 40 or 50, let alone 100. We will reduce this to Decimal(10,8) to retain precision 8 digits after the decimal point.
* 'precipitation' - is total precipitation measured in meters. As the max value is less than 1, and the rest of the measurements are all in the tens to hundreds range, we will change the unit to millimeters and the type to Float32. This will reduce the number of decimal points we are dealing with, and also put it on a similar scale as the rest of the measurements.
* 'surface_net_solar_radiation' - is the amount of solar radiation that reaches a horizontal plane at the surface of the Earth minus the amount reflected by the earth's surface. This is measured in Joules (J) per square meter (m^-2).
* 'surface_solar_radiation_downwards' - is the amount of solar radiation that reaches a horizontal plane at the surface of the Earth. This is measured in Joules (J) per square meter (m^-2).


https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

---

##### Testing Implementation of Changes

Similar to what was done with the customers data table, we will now implement the changes mentioned.

In [28]:
# We will change the location to two separate columns for longitude and latitude
# We will also convert the 'city' column to a categorical data type for better memory efficiency.
# We will then drop the unnecessary columns, and save the DataFrame to a new Parquet file()

# We'll also monitor the memory usage and time taken for this operation
# Start memory tracing
proc = psutil.Process()
mem_before = proc.memory_info().rss
tracemalloc.start()
# Start timing the conversion
t0 = time.perf_counter()

weather_lazy_operations = (
    pl.scan_parquet(weather_path)
    .with_columns([
        # converting timestamp to UTC timezone
        pl.col("timestamp").dt.replace_time_zone("UTC"),

        # converting to two separate columns for longitude and latitude
        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[0], return_dtype=pl.Float64)
          .alias("weather_longitude"),

        pl.col("location")
          .map_elements(lambda h: wkb_to_coords(h)[1], return_dtype=pl.Float64)
          .alias("weather_latitude"),

        # converting 'city' column to categorical data type
        pl.col("city").cast(pl.Categorical),

        # converting wind direction & speed to float32
        pl.col("wind_direction").cast(pl.Float32),
        pl.col("wind_speed").cast(pl.Float32),

        # converting temperature to float32
        pl.col("temperature").cast(pl.Float32),

        # converting precipitation to float 32, then to mm
        (pl.col("precipitation").cast(pl.Float32) * 1000).alias("precipitation_mm"), # precimpitation is in m, converting to mm

        # converting surface net solar radiation to Float64, then kilo-watt-hours
        ((pl.col("surface_net_solar_radiation").cast(pl.Float64) / (60 * 60))/1000).alias("surface_net_solar_radiation_kwh"),  # converting from J/m^2 to kWh/m^2
        # converting surface solar radiation downwards to Float64, then kilo-watt-hours
        ((pl.col("surface_solar_radiation_downwards").cast(pl.Float64) / (60 * 60))/1000).alias("surface_solar_radiation_downwards_kwh"),  # converting from J/m^2 to kWh/m^2
    ])
    .drop("location", "precipitation", "surface_net_solar_radiation", "surface_solar_radiation_downwards")  # drop the original versions of changed columns
    .with_columns(
        # convert to Asia/Kolkata timezone
        pl.col("timestamp").dt.replace_time_zone("Asia/Kolkata"),
))
weather_table_optimised_pldf = weather_lazy_operations.collect()

# Stop timer and memory tracing
t1 = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
mem_after = proc.memory_info().rss

# Return stats
print(f"Time taken to optimise weather data: {t1 - t0:.2f} seconds")
print(f"PyObject Memory usage [tracemalloc]: {current / (1024 * 1024):.2f} MB (peak: {peak / (1024 * 1024):.2f} MB)")
print(f"Total Physical Memory usage (RSS) [psutil]: {(mem_after - mem_before) / (2**20):.3f} MB")

Time taken to optimise weather data: 151.70 seconds
PyObject Memory usage [tracemalloc]: 0.01 MB (peak: 0.03 MB)
Total Physical Memory usage (RSS) [psutil]: 976.547 MB


In [29]:
base, ext = os.path.splitext(weather_path)
weather_table_optimised_path = f"{base}_optimised{ext}"

# Save the modified DataFrame to a new Parquet file
weather_table_optimised_pldf.write_parquet(
    weather_table_optimised_path,
    compression="snappy",  # Using snappy compression for better performance
    statistics=True,  # Enable statistics for better query performance (doesn't make a noticeable change on disk storage)
)

In [30]:
dataframe_summary(weather_table_pldf, name="Original Weather Table", filepath=weather_path)
dataframe_summary(weather_table_optimised_pldf, name="Optimized Weather Table", filepath=weather_table_optimised_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Original Weather Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 77.77 MB

Estimated size in memory: 610.27 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • location        190.24 MB  ( 31.2%)
  • wind_speed       61.35 MB  ( 10.1%)
  • wind_direction   61.35 MB  ( 10.1%)
  • temperature      61.35 MB  ( 10.1%)
  • precipitation    61.35 MB  ( 10.1%)
  • surface_net_solar_radiation  61.35 MB  ( 10.1%)
  • surface_solar_radiation_downwards  61.35 MB  ( 10.1%)
  • timestamp        30.44 MB  (  5.0%)
  • city             21.47 MB  (  3.5%)

Schema:
-----------------------------------------------------------------------------------------------------

timestamp,wind_speed,wind_direction,temperature,city,precipitation,surface_net_solar_radiation,surface_solar_radiation_downwards
datetime[μs],"decimal[38,18]","decimal[38,20]","decimal[38,15]",str,"decimal[38,28]","decimal[38,17]","decimal[38,17]"
2024-12-13 13:00:00,8.013575020999754000,105.09231811866465000000,15.975219726562500,"""delhi""",0.0000000000000000000000000000,12296608.00000000000000000,14926616.00000000000000000
2024-12-13 14:00:00,8.289883957905610000,110.72614327536141000000,14.646026611328125,"""delhi""",0.0000000000000000000000000000,12108108.00000000000000000,14736050.00000000000000000
2024-12-13 14:00:00,8.336890954411203000,110.27725102589554000000,14.629241943359375,"""delhi""",0.0000000000000000000000000000,12149601.00000000000000000,14737817.00000000000000000
2024-12-13 14:00:00,8.403726468287198000,109.70666556656998000000,14.623779296875000,"""delhi""",0.0000000000000000000000000000,12240716.00000000000000000,14738703.00000000000000000
2024-12-13 14:00:00,8.454637190808407000,109.04352821446052000000,14.611297607421875,"""delhi""",0.0000000000000000000000000000,12278334.00000000000000000,14740205.00000000000000000


Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Optimized Weather Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 55.57 MB

Estimated size in memory: 231.14 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • surface_net_solar_radiation_kwh  30.91 MB  ( 13.4%)
  • surface_solar_radiation_downwards_kwh  30.91 MB  ( 13.4%)
  • timestamp        30.44 MB  ( 13.2%)
  • weather_longitude  30.44 MB  ( 13.2%)
  • weather_latitude  30.44 MB  ( 13.2%)
  • wind_speed       15.69 MB  (  6.8%)
  • wind_direction   15.69 MB  (  6.8%)
  • temperature      15.69 MB  (  6.8%)
  • precipitation_mm  15.69 MB  (  6.8%)
  • 

timestamp,wind_speed,wind_direction,temperature,city,weather_longitude,weather_latitude,precipitation_mm,surface_net_solar_radiation_kwh,surface_solar_radiation_downwards_kwh
"datetime[μs, Asia/Kolkata]",f32,f32,f32,cat,f64,f64,f32,f64,f64
2024-12-13 13:00:00 IST,8.013575,105.092316,15.97522,"""delhi""",77.34,28.4,0.0,3.415724,4.146282
2024-12-13 14:00:00 IST,8.289884,110.726143,14.646027,"""delhi""",76.84,28.8,0.0,3.363363,4.093347
2024-12-13 14:00:00 IST,8.336891,110.277252,14.629242,"""delhi""",76.94,28.8,0.0,3.374889,4.093838
2024-12-13 14:00:00 IST,8.403727,109.706665,14.623779,"""delhi""",77.04,28.8,0.0,3.400199,4.094084
2024-12-13 14:00:00 IST,8.454638,109.043526,14.611298,"""delhi""",77.14,28.8,0.0,3.410648,4.094501


Note: id and location related columns are redacted here to preserve customer confidentiality


As we can see, both the file size on disk and the file size in memory has been reduced by these changes.

In [31]:
# removing the temporary files created during the process
try:
    os.remove(weather_table_optimised_path)
    print(f"Successfully deleted: {weather_table_optimised_path}")
except FileNotFoundError:
    print(f"File does not exist: {weather_table_optimised_path}")


Successfully deleted: ../../data/hitachi/weather_20250714_1401_optimised.parquet


#### Grid readings table [grid_readings_table_pldf]

In [32]:
# Loading the data
grid_readings_table_pldf = pl.read_parquet(grid_readings_path)


In [33]:
dataframe_summary(grid_readings_table_pldf, name="Grid Readings Table", filepath=grid_readings_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Grid Readings Table 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 21.71 MB

Estimated size in memory: 51.37 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • timestamp         4.28 MB  (  8.3%)
  • thermal_generation   4.28 MB  (  8.3%)
  • gas_generation    4.28 MB  (  8.3%)
  • g_co2_per_kwh     4.28 MB  (  8.3%)
  • hydro_generation   4.28 MB  (  8.3%)
  • nuclear_generation   4.28 MB  (  8.3%)
  • renewable_generation   4.28 MB  (  8.3%)
  • tons_co2          4.28 MB  (  8.3%)
  • total_generation   4.28 MB  (  8.3%)
  • tons_co2_per_mwh   4.28 MB  (  8.3%)
  • demand_met        4.28 MB  (  8.3%)
  • net_demand        4.28 MB  (  8.3%)

Schema:
-

timestamp,thermal_generation,gas_generation,g_co2_per_kwh,hydro_generation,nuclear_generation,renewable_generation,tons_co2,total_generation,tons_co2_per_mwh,demand_met,net_demand
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2022-01-04 17:05:00,136202.0,2901.0,815.573088,10113.0,5211.0,9900.0,11168.3899,164327.0,0.815573,162948.0,154427.0
2022-01-04 17:10:00,136202.0,2901.0,815.573088,10113.0,5211.0,9900.0,11168.3899,164327.0,0.815573,162948.0,154427.0
2022-01-04 17:15:00,136202.0,2901.0,815.573088,10113.0,5211.0,9900.0,11168.3899,164327.0,0.815573,162948.0,154427.0
2022-01-04 17:20:00,136202.0,2901.0,815.573088,10113.0,5211.0,9900.0,11168.3899,164327.0,0.815573,162948.0,154427.0
2022-01-04 17:25:00,136202.0,2901.0,815.573088,10113.0,5211.0,9900.0,11168.3899,164327.0,0.815573,162948.0,154427.0


Note: id and location related columns are redacted here to preserve customer confidentiality


We could potentially downcast some of these larger-valued columns to float32, but at this point we might as well leave them. They are not taking up very much space in memory or in disk at all and will take up even less after aggregated to the hourly level.

#### Meter Readings Datasets

##### Large Meter Readings Dataset [meter_readings_2021_pldf]

In [34]:
# Loading the data
meter_readings_2021_pldf = pl.read_parquet(meter_readings_2021_path)

In [35]:
dataframe_summary(meter_readings_2021_pldf, name="Meter Readings Table (2021)", filepath=meter_readings_2021_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Meter Readings Table (2021) 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 2,080.60 MB

Estimated size in memory: 9,277.18 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • date            4406.66 MB  ( 47.5%)
  • ca_id           1855.44 MB  ( 20.0%)
  • value           1855.44 MB  ( 20.0%)
  • city            1159.65 MB  ( 12.5%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • ca_id: Int64
  • date: String
  • value: Float64
  • city: String

First 5 rows: 
--------------------------------------------------------------------------------------------------------------

date,city
str,str
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""


Note: id and location related columns are redacted here to preserve customer confidentiality


In [36]:
# Lets change date to datetime, ca_id to string, and city to categorical

meter_readings_2021_lazy_operations = (
    pl.scan_parquet(meter_readings_2021_path)
    .with_columns([
    # converting 'date' column to datetime YYYY-MM-DD HH:MM:SS and setting timezone to Asia/Kolkata
        (pl.col("date").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S")).dt.replace_time_zone("Asia/Kolkata"),
        # converting 'ca_id' column to string
        pl.col("ca_id").cast(pl.String),
        # converting 'city' column to categorical
        pl.col("city").cast(pl.Categorical),
    ])
)
meter_readings_2021_optimised_pldf = meter_readings_2021_lazy_operations.collect()

# Save the modified DataFrame to a new Parquet file
base, ext = os.path.splitext(meter_readings_2021_path)
meter_readings_2021_optimised_path = f"{base}_optimised{ext}"

meter_readings_2021_optimised_pldf.write_parquet(
    meter_readings_2021_optimised_path,
    compression="snappy",  # Using snappy compression for better performance
    statistics=True,  # Enable statistics for better query performance (doesn't make a huge impact on disk storage)
)

dataframe_summary(meter_readings_2021_pldf, name="Original Meter Readings Table (2021)", filepath=meter_readings_2021_path)
dataframe_summary(meter_readings_2021_optimised_pldf, name="Optimized Meter Readings Table (2021)", filepath=meter_readings_2021_optimised_path)


------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Original Meter Readings Table (2021) 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 2,080.60 MB

Estimated size in memory: 9,277.18 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • date            4406.66 MB  ( 47.5%)
  • ca_id           1855.44 MB  ( 20.0%)
  • value           1855.44 MB  ( 20.0%)
  • city            1159.65 MB  ( 12.5%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • ca_id: Int64
  • date: String
  • value: Float64
  • city: String

First 5 rows: 
-----------------------------------------------------------------------------------------------------

date,city
str,str
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""
"""2021-12-01 00:00:00""","""delhi"""


Note: id and location related columns are redacted here to preserve customer confidentiality

------------------------------------------------------------------------------------------------------------------------
DataFrame Summary — Optimized Meter Readings Table (2021) 
------------------------------------------------------------------------------------------------------------------------

File size on disk: 2,414.18 MB

Estimated size in memory: 7,189.82 MB

Per-column memory usage:
------------------------------------------------------------------------------------------------------------------------
  • ca_id           2551.23 MB  ( 35.5%)
  • date            1855.44 MB  ( 25.8%)
  • value           1855.44 MB  ( 25.8%)
  • city            927.72 MB  ( 12.9%)

Schema:
------------------------------------------------------------------------------------------------------------------------
  • ca_id: String
  • date: Datetime(time_unit='us', time_zone='Asia/Kolkata')
  • value: Floa

date,city
"datetime[μs, Asia/Kolkata]",cat
2021-12-01 00:00:00 IST,"""delhi"""
2021-12-01 00:00:00 IST,"""delhi"""
2021-12-01 00:00:00 IST,"""delhi"""
2021-12-01 00:00:00 IST,"""delhi"""
2021-12-01 00:00:00 IST,"""delhi"""


Note: id and location related columns are redacted here to preserve customer confidentiality


We may have increased the size of the file on disk, but we have reduced the size in memory.
Additionally, the file should be easier and quicker to query with the statistics saving enabled on polars.

Though the ca_id column increased significantly in size, we are now using string format instead of integer, which is recommended for ids.

In [37]:
# Removing the temporary files created during the process
try:
    os.remove(meter_readings_2021_optimised_path)
    print(f"Successfully deleted: {meter_readings_2021_optimised_path}")
except FileNotFoundError:
    print(f"File does not exist: {meter_readings_2021_optimised_path}")

Successfully deleted: ../../data/hitachi/meter_primary_files/meter_readings_2021_20250714_2015_optimised.parquet


## *Summary*

**KEY TAKEAWAYS:** 
* Use parquet files instead of csv files
* Use snappy compression for better performance
* Save the file with statistics enabled to allow for faster queries
* Use polars' lazy execution for better performance with large datasets
* Use the correct data types for each column to reduce memory usage

**Related changes:**
* Make sure to convert timestamps to the correct timezone
* Change city column to categorical datatype
* Change location column to two separate columns for longitude and latitude
* Change wind_direction, wind_speed, and temperature to float32
* Change precipitation to millimeters and float32
* Change solar radiation fields to kWh/m^2 and float32